In [19]:
import pandas as pd
import numpy as np
pd.options.display.float_format = '{:.5f}'.format

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler
from scipy.stats.mstats import winsorize              # 극단값 조정에 사용
from matplotlib import font_manager, rc
font_path = "C:/Windows/Fonts/malgun.ttf"
font = font_manager.FontProperties(fname=font_path).get_name()
rc('font', family=font)
import warnings
warnings.filterwarnings(action = 'ignore')

#### 데이터셋 불러오기

In [136]:
kosdaq1 = pd.read_csv('./dataset/KOSDAQ_연결.csv', encoding='utf-8')
kosdaq2 = pd.read_csv('./dataset/KOSDAQ_개별.csv', encoding='utf-8')

kosdaq = pd.merge(kosdaq1, kosdaq2, on=['회사명', '거래소코드', '회계년도'])

kosdaq1['거래소코드'] = kosdaq1['거래소코드'].astype(str).str.zfill(6)
kosdaq2['거래소코드'] = kosdaq2['거래소코드'].astype(str).str.zfill(6)
kosdaq['거래소코드'] = kosdaq['거래소코드'].astype(str).str.zfill(6)

In [138]:
kospi1 = pd.read_csv('./dataset/KOSPI_연결.csv', encoding='utf-8')
kospi2 = pd.read_csv('./dataset/KOSPI_개별.csv', encoding='utf-8')

kospi = pd.merge(kospi1, kospi2, on=['회사명', '거래소코드', '회계년도'])

kospi1['거래소코드'] = kospi1['거래소코드'].astype(str).str.zfill(6)
kospi2['거래소코드'] = kospi2['거래소코드'].astype(str).str.zfill(6)
kospi['거래소코드'] = kospi['거래소코드'].astype(str).str.zfill(6)

In [139]:
kosdaq_bankruptcy = pd.read_csv('./dataset/KOSDAQ_부실.csv', encoding='utf-8')
kospi_bankruptcy = pd.read_csv('./dataset/KOSPI_부실.csv', encoding='utf-8')

kosdaq_bankruptcy['거래소코드'] = kosdaq_bankruptcy['거래소코드'].astype(str).str.zfill(6)
kospi_bankruptcy['거래소코드'] = kospi_bankruptcy['거래소코드'].astype(str).str.zfill(6)

#### 회사, 거래소코드 확인

- 같은 회사명에 거래소 코드 여러개
- 같은 거래소 코드에 회사명 여러개

In [140]:
print('코스닥')
print('회사', len(kosdaq['회사명'].unique()))
print('코드', len(kosdaq['거래소코드'].unique()))
print('\n코스피')
print('회사', len(kospi['회사명'].unique()))
print('코드', len(kospi['거래소코드'].unique()))

코스닥
회사 1727
코드 1745

코스피
회사 812
코드 815


In [141]:
## 회사명을 기준으로 거래소코드를 추출할 경우 
## 거래소코드의 갯수를 확인하는 코드
daq = kosdaq.groupby(['회사명'])['거래소코드'].nunique().reset_index()

# daq 데이터프레임 중에 거래소코드 갯수가 1개 이상인 회사명 추출
daq[daq['거래소코드']>1]

,회사명,거래소코드
281,(주)본느,2
377,(주)세화피앤씨,2
498,(주)아이엘사이언스,2
524,(주)알로이스,2
652,(주)엔케이맥스,2
844,(주)인산가,2
867,(주)자비스,2
872,(주)정다운,2
1118,(주)판타지오,2
1134,(주)포인트엔지니어링,2


In [142]:
pi = kospi.groupby(['회사명'])['거래소코드'].nunique().reset_index()
pi[pi['거래소코드']>1]

,회사명,거래소코드
143,(주)신성이엔지,2
505,삼성물산(주),2
582,에스케이(주),2


In [143]:
col = pd.Series(kosdaq1.columns) 
col

0                  회사명
1                거래소코드
2                 회계년도
3              유동자산_연결
4             비유동자산_연결
            ...       
82    매입채무 및 기타유동채무_연결
83            단기차입금_연결
84            장기차입금_연결
85               사채_연결
86             이자비용_연결
Length: 87, dtype: object

#### 연결 + 개별 DataFrame 합치기

In [144]:
# 코스닥1의 컬럼명을 시리즈형태로 구성
col = pd.Series(kosdaq1.columns)          

# split : 설정한 기준(언더바)를 기준으로 데이터의 값을 구분해줌
# get   :
col = col.str.split('_').str.get(0)[3:]   

col = col.reset_index(drop=True)

# 연결에 없는 값 개별에 있으면 대입
for i in range(len(col)):
        kosdaq.loc[kosdaq[col[i]+'_연결'].isna(), col[i]+'_연결'] = kosdaq[col[i]+'_개별']
# loc : 행 을 조회
# loc[kosdaq[col[i]+'_연결'].isna() : 해당 조건을 만족하는 행을 출력 요청
# kosdaq[col[i]+'_개별'] : 위에서 불러온 행의 값(=null 값) 을 현재의 조건으로 변환 

# 컬럼 정리
kosdaq.drop(list(kosdaq2.columns)[3:], axis=1, inplace=True)
col_name = pd.Series(kosdaq.columns)
kosdaq.columns = col_name.str.split('_').str.get(0)

In [145]:
col = pd.Series(kospi1.columns)
col = col.str.split('_').str.get(0)[3:]
col = col.reset_index(drop=True)

# 연결에 없는 값 개별에 있으면 대입
for i in range(len(col)):
        kospi.loc[kospi[col[i]+'_연결'].isna(), col[i]+'_연결'] = kospi[col[i]+'_개별']

# 컬럼 정리
kospi.drop(list(kospi2.columns)[3:], axis=1, inplace=True)
col_name = pd.Series(kospi.columns)
kospi.columns = col_name.str.split('_').str.get(0)

#### 회계년도, 회계월 정리

In [146]:
kosdaq

,회사명,거래소코드,회계년도,유동자산,비유동자산,유동부채,비유동부채,자본금,자본잉여금,기타포괄손익누계액,...,유형자산회전율,순운전자본회전률,1회전기간,종업원수,재고자산,매입채무 및 기타유동채무,단기차입금,장기차입금,사채,이자비용
0,(주)CMG제약,058820,2011/12,15341.00000,11964.00000,7634.00000,1637.00000,21847.00000,39418.00000,41.00000,...,1.77000,0.86000,0.01000,135.00000,2743.00000,3386.00000,0.00000,0.00000,478.00000,181.00000
1,(주)CMG제약,058820,2012/12,47109.00000,9999.00000,6793.00000,18724.00000,33013.00000,50959.00000,-1.00000,...,1.71000,0.42000,0.00000,134.00000,2955.00000,4414.00000,100.00000,0.00000,17362.00000,543.00000
2,(주)CMG제약,058820,2013/12,38210.00000,17703.00000,5500.00000,1415.00000,43313.00000,57682.00000,-1.00000,...,2.03000,0.42000,0.01000,136.00000,4619.00000,5423.00000,0.00000,0.00000,1415.00000,1134.00000
3,(주)CMG제약,058820,2014/12,28301.00000,29767.00000,8719.00000,346.00000,43313.00000,57682.00000,-1.00000,...,1.54000,0.56000,0.00000,164.00000,4913.00000,7034.00000,5.00000,0.00000,0.00000,123.00000
4,(주)CMG제약,058820,2015/12,26272.00000,30261.00000,8850.00000,876.00000,44149.00000,58464.00000,-1.00000,...,1.48000,0.75000,0.01000,156.00000,5456.00000,8668.00000,6.00000,0.00000,0.00000,80.00000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12290,흥구석유(주),024060,2015/12,13780.00000,64274.00000,2734.00000,7093.00000,1500.00000,35534.00000,0.00000,...,3.83000,8.87000,0.06000,99.00000,1652.00000,1098.00000,0.00000,5000.00000,0.00000,55.00000
12291,흥구석유(주),024060,2016/12,14794.00000,64344.00000,3594.00000,6366.00000,1500.00000,35534.00000,0.00000,...,3.67000,8.24000,0.06000,96.00000,2519.00000,1806.00000,1000.00000,5000.00000,0.00000,126.00000
12292,흥구석유(주),024060,2017/12,15880.00000,64032.00000,8946.00000,1211.00000,1500.00000,35534.00000,45.00000,...,4.10000,7.20000,0.05000,93.00000,2756.00000,2065.00000,6000.00000,0.00000,0.00000,127.00000
12293,흥구석유(주),024060,2018/12,12515.00000,62691.00000,3802.00000,961.00000,1500.00000,35534.00000,0.00000,...,4.40000,8.08000,0.06000,84.00000,1486.00000,2343.00000,0.00000,0.00000,0.00000,86.00000


In [147]:
kosdaq.loc[kosdaq['회계년도']=='2019', '회계년도'] = '2019/12'
kosdaq['회계월'] = kosdaq['회계년도'].str.split('/').str.get(1)
kosdaq['회계년도'] = kosdaq['회계년도'].str.split('/').str.get(0).astype('int64')
kosdaq['회계년도+1'] = kosdaq['회계년도']+1

In [148]:
kospi.loc[kospi['회계년도']=='2019', '회계년도'] = '2019/12'
kospi['회계월'] = kospi['회계년도'].str.split('/').str.get(1)
kospi['회계년도'] = kospi['회계년도'].str.split('/').str.get(0).astype('int64')
kospi['회계년도+1'] = kospi['회계년도']+1

In [149]:
len(kospi['거래소코드'].unique())

815

#### 부실 기업 (Target)

##### 코스피

In [150]:
# left_on : 왼쪽 데이터프레임의 기준은 ['회계년도+1', '거래소코드'] 이고
# right_on : 오른쪽 데이터프레임의 기준은 ['회계년도', '거래소코드']

kospi = pd.merge(kospi, kospi_bankruptcy[['회계년도', '거래소코드']], 
                 left_on=['회계년도+1', '거래소코드'],                   
                 right_on=['회계년도', '거래소코드'], 
                 how='left')
kospi.head()
# 위 코드를 실행하면 회계년도가 여러개여서 회계년도_x, 회계년도_y 가 추가로 생성됨

,회사명,거래소코드,회계년도_x,유동자산,비유동자산,유동부채,비유동부채,자본금,자본잉여금,기타포괄손익누계액,...,종업원수,재고자산,매입채무 및 기타유동채무,단기차입금,장기차입금,사채,이자비용,회계월,회계년도+1,회계년도_y
0,(주)DB하이텍,000990,2011,279726.00000,919318.00000,794705.00000,96277.00000,214616.00000,684522.00000,2709.00000,...,2567.00000,77113.00000,61197.00000,5398.00000,0.00000,0.00000,57547.00000,12,2012,NaN
1,(주)DB하이텍,000990,2012,309144.00000,848611.00000,555433.00000,311428.00000,222556.00000,691522.00000,3264.00000,...,2304.00000,79262.00000,63776.00000,5142.00000,214817.00000,0.00000,50197.00000,12,2013,NaN
2,(주)DB하이텍,000990,2013,222961.00000,835431.00000,476868.00000,376767.00000,222556.00000,128091.00000,-1418.00000,...,2142.00000,72378.00000,67568.00000,6452.00000,276847.00000,0.00000,59545.00000,12,2014,NaN
3,(주)DB하이텍,000990,2014,247582.00000,737294.00000,376437.00000,487737.00000,222556.00000,128092.00000,-4315.00000,...,1999.00000,69180.00000,91825.00000,7547.00000,391620.00000,0.00000,67651.00000,12,2015,NaN
4,(주)DB하이텍,000990,2015,337757.00000,646886.00000,244045.00000,486541.00000,222556.00000,128092.00000,-8855.00000,...,2004.00000,83460.00000,98490.00000,5219.00000,399749.00000,0.00000,50040.00000,12,2016,NaN


In [151]:
# 회계년도 x 가 실제 회계년도이므로 컬럼명 수정해주고, 
# 회계년도 y 는 부실기업의 회계년도이고, 정상기업인 경우는 회계년도 데이터가 없을 것이므로 NaN 값이 생성될 것.. 
kospi.rename(columns={'회계년도_x':'회계년도',
                    '회계년도_y':'target'}, inplace=True)

# target 에 NaN 값이 있으면 0, 정상기업이면 1 을 채워줌
kospi['target'] = np.where(kospi['target'].isna(), 0, 1)
kospi.head()

,회사명,거래소코드,회계년도,유동자산,비유동자산,유동부채,비유동부채,자본금,자본잉여금,기타포괄손익누계액,...,종업원수,재고자산,매입채무 및 기타유동채무,단기차입금,장기차입금,사채,이자비용,회계월,회계년도+1,target
0,(주)DB하이텍,000990,2011,279726.00000,919318.00000,794705.00000,96277.00000,214616.00000,684522.00000,2709.00000,...,2567.00000,77113.00000,61197.00000,5398.00000,0.00000,0.00000,57547.00000,12,2012,0
1,(주)DB하이텍,000990,2012,309144.00000,848611.00000,555433.00000,311428.00000,222556.00000,691522.00000,3264.00000,...,2304.00000,79262.00000,63776.00000,5142.00000,214817.00000,0.00000,50197.00000,12,2013,0
2,(주)DB하이텍,000990,2013,222961.00000,835431.00000,476868.00000,376767.00000,222556.00000,128091.00000,-1418.00000,...,2142.00000,72378.00000,67568.00000,6452.00000,276847.00000,0.00000,59545.00000,12,2014,0
3,(주)DB하이텍,000990,2014,247582.00000,737294.00000,376437.00000,487737.00000,222556.00000,128092.00000,-4315.00000,...,1999.00000,69180.00000,91825.00000,7547.00000,391620.00000,0.00000,67651.00000,12,2015,0
4,(주)DB하이텍,000990,2015,337757.00000,646886.00000,244045.00000,486541.00000,222556.00000,128092.00000,-8855.00000,...,2004.00000,83460.00000,98490.00000,5219.00000,399749.00000,0.00000,50040.00000,12,2016,0


In [152]:
# 정상 및 부실기업 확인
print(len(kospi[kospi['target']==0]))
print(len(kospi[kospi['target']==1]))

5987
591


코스닥

In [153]:
kosdaq = pd.merge(kosdaq, kosdaq_bankruptcy[['회계년도', '거래소코드']], left_on=['회계년도+1', '거래소코드'], right_on=['회계년도', '거래소코드'], how='left')
kosdaq.rename(columns={'회계년도_x':'회계년도',
                    '회계년도_y':'target'}, inplace=True)
kosdaq['target'] = np.where(kosdaq['target'].isna(), 0, 1)

In [154]:
print(len(kosdaq[kosdaq['target']==0]))
print(len(kosdaq[kosdaq['target']==1]))

11026
1269


#### 회사 drop 함수

In [155]:
# 회사 drop 할 함수 만들어두기
def drop_company(df, list):
    df.set_index('거래소코드', drop=True, inplace=True)
    df.drop(list, axis=0, inplace=True)
    df.reset_index(inplace=True)

#### 코스닥 中 연구개발업종 + 특례 상장 제외

In [156]:
dropdf = pd.read_csv('./dataset/연구개발업.csv', encoding='utf-8')
dropdf['거래소코드'] = dropdf['거래소코드'].astype(str).str.zfill(6)

droplist = dropdf['거래소코드'].unique()                                    # 연구개발업종
droplist2 = ['263050', '208340', '288330', '294090', '334970', '226610']    # 특례 상장 (+한국비엔씨)
droplist = np.concatenate([droplist, droplist2])

In [39]:
# (주)유틸렉스 : 263050
# (주)파멥신 : 208340
# 브릿지바이오테라퓨틱스(주) : 288330
# 이오플로우(주) : 294090
# 프레스티지바이오로직스(주) : 334970

In [157]:
drop_company(kosdaq, droplist)

In [158]:
kosdaq.columns[kosdaq.columns.str.contains('매출')]

Index(['매출액', '매출원가', '매출총이익', '매출채권 및 기타유동채권', '매출액증가율', '매출액총이익률', '매출채권비율',
       '매출채권 대 매입채무비율', 'CASH FLOW 대 매출액비율', '매출채권회전률', '매출채권회전기간'],
      dtype='object')

In [160]:
# 외부 감사인의 감사, 검토를 받지 않아서 결측치 존재 -> 제거
kosdaq.drop(kosdaq[(kosdaq['회사명']=='우리손에프앤지농업회사법인(주)') & (kosdaq['회계년도']==2011)].index, axis=0, inplace=True)

# 회사설립일 & I/S에 따른 결측치 존재 -> 제거
kosdaq.drop(kosdaq[(kosdaq['회사명']=='(주)슈프리마') & (kosdaq['회계년도']==2015)].index, axis=0, inplace=True)
kosdaq.drop(kosdaq[(kosdaq['회사명']=='덕산네오룩스(주)') & (kosdaq['회계년도']==2014)].index, axis=0, inplace=True)


#### 코스피 - 기타법인 제거

In [161]:
droplist = ['104110', '000830', '003600']
drop_company(kospi, droplist)

#### 결측치 처리

##### 1. 재무제표를 못 불러오는 row 제거

In [162]:
kosdaq.dropna(subset=['유형자산'], axis=0, inplace=True)

In [163]:
kospi.dropna(subset=['유형자산'], axis=0, inplace=True)

##### 2. 회계년도가 하나인 기업 제거

M Score -> 작년과 올해의 비율을 계산해야하기 때문에 회계년도가 하나이면 계산 불가능

In [164]:
# 거래소 코드를 기준으로 그룹화해서 회계년도 갯수를 count
company_d_cnt = kosdaq.groupby('거래소코드').count()['회계년도']

# 회계년도 갯수가 1개인 인덱스 추출
company_d_cnt[company_d_cnt==1].index

Index(['002670', '007150', '015200', '015390', '019260', '033280', '034010',
       '035210', '037320', '038120', '038320', '038990', '040740', '042340',
       '042820', '043890', '045050', '045260', '047730', '048460', '050050',
       '050470', '052350', '053040', '056010', '056710', '060850', '065310',
       '065610', '066690', '067130', '068630', '071660', '073130', '086710',
       '089480', '094700', '098400', '105070', '126340', '129890', '136410',
       '160350', '163730', '167380', '225220', '227950', '230400', '233190',
       '239340', '242420', '250930', '253590', '254120', '256090', '256630',
       '256840', '258790', '262840', '264850', '271400', '297570', '299030',
       '303030', '307180', '314130', '315640', '318410', '319660', '321820',
       '330860', '331920', '333620', '337930', '338220', '340570', '340930',
       '347000', '347770', '347860', '347890', '348350', '348370', '351330',
       '352480', '352770', '352910', '352940', '354200', '357550', '361570',

In [165]:
drop_company(kosdaq, company_d_cnt[company_d_cnt==1].index)

In [166]:
company_p_cnt = kospi.groupby('거래소코드').count()['회계년도']
company_p_cnt[company_p_cnt==1].index

Index(['001310', '004010', '004550', '009380', '011800', '012400', '014300',
       '015110', '016570', '026870', '064420', '244920', '329180', '336260',
       '336370', '352820', '361610', '381970', '900030'],
      dtype='object', name='거래소코드')

In [167]:
drop_company(kospi, company_p_cnt[company_p_cnt==1].index)

##### 3. 회계월 변경 기업 제거

코스닥

In [169]:
# 거래소코드 별 회계월 별로 그룹화 후 중복값 제거
kosdaq_month = kosdaq.groupby(['거래소코드'])['회계월'].unique()

diff_kosdaq = []
for i in range(len(kosdaq_month)):
    if len(kosdaq_month.values[i]) != 1:
        diff_kosdaq.append(kosdaq_month.index[i])

In [170]:
kosdaq.set_index('거래소코드', drop=True, inplace=True)
kosdaq.drop(diff_kosdaq, axis=0, inplace=True)
kosdaq.drop('회계월', axis=1, inplace=True)
kosdaq.reset_index(inplace=True)

코스피

In [172]:
kospi_month = kospi.groupby(['거래소코드'])['회계월'].unique()

diff_kospi = []
for i in range(len(kospi_month)):
    if len(kospi_month.values[i]) != 1:
        diff_kospi.append(kospi_month.index[i])

In [173]:
kospi.set_index('거래소코드', drop=True, inplace=True)
kospi.drop(diff_kospi, axis=0, inplace=True)
kospi.drop('회계월', axis=1, inplace=True)
kospi.reset_index(inplace=True)

##### 4. 감가상각비

감가상각비가 NaN거나, 0.0일 때

주석에 감가상각비 값이 들어있으면 대입

그래도 없으면 dropna

In [174]:
kosdaq.loc[kosdaq['감가상각비'].isna(), '감가상각비'] = kosdaq['감가(주석)']/1000
kospi.loc[kospi['감가상각비'].isna(), '감가상각비'] = kospi['감가(주석)']/1000

kosdaq.loc[kosdaq['감가상각비']==0.0, '감가상각비'] = kosdaq['감가(주석)']/1000
kospi.loc[kospi['감가상각비']==0.0, '감가상각비'] = kospi['감가(주석)']/1000

In [175]:
kosdaq.drop(['감가(주석)'], axis=1, inplace=True)
kospi.drop(['감가(주석)'], axis=1, inplace=True)

감가상각비가 결측치면 0.0으로 뜨게 된다.

감가상각비가 0.0인 row -> 그룹별 평균으로 채우기

In [176]:
# 감가상각비가 0이면 0을 결측치로 채워줌
kosdaq.loc[kosdaq['감가상각비']==0.0, '감가상각비'] = np.nan

# 기업 별 전체 감가상각비 평균값으로 0 을 대체해줌 
# transform : nan 값이 포함되어 있어서 그냥 mean 을 쓰면 에러나서.. transform 사용
kosdaq['감가상각비'] = kosdaq['감가상각비'].fillna(kosdaq.groupby('거래소코드')['감가상각비'].transform('mean'))

In [177]:
kospi.loc[kospi['감가상각비']==0.0, '감가상각비'] = np.nan
kospi['감가상각비'] = kospi['감가상각비'].fillna(kospi.groupby('거래소코드')['감가상각비'].transform('mean'))

지에스는 모든 년도에 감가상각비가 없음(지주사이기 때문에)

지에스 감가상각비 모두 0.0으로 채우기

In [178]:
kospi[(kospi['감가상각비'].isna())][['회사명', '거래소코드', '회계년도', '감가상각비']].sort_values('회계년도')

,회사명,거래소코드,회계년도,감가상각비
1976,(주)지에스,078930,2011,NaN
1974,(주)지에스,078930,2012,NaN
1975,(주)지에스,078930,2013,NaN
1972,(주)지에스,078930,2014,NaN
1968,(주)지에스,078930,2015,NaN
1970,(주)지에스,078930,2016,NaN
1971,(주)지에스,078930,2017,NaN
1973,(주)지에스,078930,2018,NaN
1969,(주)지에스,078930,2019,NaN


In [59]:
kospi['감가상각비'] = kospi['감가상각비'].fillna(0.0)

##### 5. 파생변수 계산 + NaN 처리

코스닥

In [179]:
kosdaq['총자본증가율'] = (kosdaq['자산'] - kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['자산'].shift(1)) / kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['자산'].shift(1) * 100

kosdaq['유형자산증가율'] = (kosdaq['유형자산'] - kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유형자산'].shift(1)) / kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유형자산'].shift(1) * 100

kosdaq['비유동자산증가율'] = (kosdaq['비유동자산'] - kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1)) / kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1) * 100

kosdaq['유동자산증가율'] = (kosdaq['유동자산'] - kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유동자산'].shift(1)) / kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유동자산'].shift(1) * 100

kosdaq['매출액증가율'] = (kosdaq['매출액'] - kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매출액'].shift(1)) / kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매출액'].shift(1) * 100

kosdaq['매출액총이익률'] = kosdaq['매출총이익']/kosdaq['매출액']*100
kosdaq['유동비율'] = kosdaq['유동자산']/kosdaq['자산']*100
kosdaq['유동자산 대 비유동자산비율'] = kosdaq['유동자산']/kosdaq['비유동자산']*100
kosdaq['비유동자산구성비율'] = kosdaq['비유동자산']/kosdaq['자산']*100
kosdaq['자기자본구성비율'] = kosdaq['자본']/kosdaq['자산']*100
kosdaq['타인자본구성비율'] = kosdaq['부채']/kosdaq['자산']*100
kosdaq['비유동비율'] = kosdaq['비유동자산']/kosdaq['자본']*100
kosdaq['비유동장기적합률'] = kosdaq['비유동자산']/(kosdaq['부채']-kosdaq['유동부채']+kosdaq['자본'])*100
kosdaq['유동비율'] = kosdaq['유동자산']/kosdaq['유동부채']*100
kosdaq['현금비율'] = kosdaq['현금및현금성자산']/kosdaq['자산']*100
kosdaq['매출채권비율'] = kosdaq['매출채권 및 기타유동채권']/kosdaq['자산']*100
kosdaq['순운전자본'] = kosdaq['유동자산']-kosdaq['유동부채']
kosdaq['부채비율'] = kosdaq['부채']/kosdaq['자본']*100
kosdaq['유동부채비율'] = kosdaq['유동부채']/kosdaq['자본']*100
kosdaq['비유동부채비율'] = (kosdaq['부채']-kosdaq['유동부채'])/kosdaq['자본']*100
kosdaq['비유동부채 대 순운전자본비율'] = (kosdaq['부채']-kosdaq['유동부채'])/kosdaq['순운전자본']*100
kosdaq['순운전자본비율'] = kosdaq['순운전자본']/kosdaq['자산']*100
kosdaq['차입금의존도'] = (kosdaq['장기차입금']+kosdaq['단기차입금']+kosdaq['사채'])/kosdaq['자산']*100
kosdaq['차입금비율'] = (kosdaq['장기차입금']+kosdaq['단기차입금']+kosdaq['사채'])/kosdaq['자본']*100
kosdaq['이자보상배율(이자비용)'] = kosdaq['영업손익']/kosdaq['이자비용']
kosdaq['CASH FLOW 대 부채비율'] = kosdaq['OCF']/kosdaq['부채']*100
kosdaq['CASH FLOW 대 차입금비율'] = kosdaq['OCF']/(kosdaq['장기차입금']+kosdaq['단기차입금']+kosdaq['사채'])*100
kosdaq['CASH FLOW 대 매출액비율'] = kosdaq['OCF']/kosdaq['매출액']*100
kosdaq['총자본회전률'] = kosdaq['매출액']/kosdaq['자산']*100

kosdaq['자기자본회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['자본'].shift(1) + kosdaq['자본'])*100
kosdaq['매입채무회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매입채무 및 기타유동채무'].shift(1) + kosdaq['매입채무 및 기타유동채무'])*100
kosdaq['매입채무회전기간'] = 365/kosdaq['매입채무회전률']*100
kosdaq['유동자산회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유동자산'].shift(1) + kosdaq['유동자산'])*100
kosdaq['당좌자산'] = kosdaq['유동자산']-kosdaq['재고자산']
kosdaq['당좌자산회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['당좌자산'].shift(1) + kosdaq['당좌자산'])*100
kosdaq['재고자산회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['재고자산'].shift(1) + kosdaq['재고자산'])*100
kosdaq['재고자산회전기간'] = 365/kosdaq['재고자산회전률']*100
kosdaq['매출채권회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매출채권 및 기타유동채권'].shift(1) + kosdaq['매출채권 및 기타유동채권'])*100
kosdaq['매출채권회전기간'] = 365/kosdaq['매출채권회전률']*100
kosdaq['비유동자산회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1) + kosdaq['비유동자산'])*100
kosdaq['유형자산회전율'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유형자산'].shift(1) + kosdaq['유형자산'])*100
kosdaq['비유동자산회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1) + kosdaq['비유동자산'])*100
kosdaq['순운전자본회전률'] = kosdaq['매출액']/(kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['순운전자본'].shift(1) + kosdaq['순운전자본'])*100
kosdaq['1회전기간'] = kosdaq['매출채권회전기간'] + kosdaq['재고자산회전기간'] - kosdaq['매입채무회전기간']
kosdaq['당좌자산구성비율'] = kosdaq['당좌자산']/kosdaq['자산']*100
kosdaq['당좌비율'] = kosdaq['당좌자산']/kosdaq['유동부채']*100
kosdaq['재고자산 대 순운전자본비율'] = kosdaq['재고자산']/kosdaq['순운전자본']*100
kosdaq['재고자산증가율'] = (kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['재고자산'].shift(1) + kosdaq['재고자산'])/kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['재고자산'].shift(1)*100
kosdaq['매출채권 대 매입채무비율'] = kosdaq['매출채권 및 기타유동채권']/kosdaq['매입채무 및 기타유동채무']*100
kosdaq['매입채무 대 재고자산비율'] = kosdaq['매입채무 및 기타유동채무']/kosdaq['재고자산']*100
kosdaq['단기차입금 대 총차입금비율'] = kosdaq['단기차입금']/(kosdaq['단기차입금']+kosdaq['장기차입금']+kosdaq['사채'])*100
kosdaq['매출액영업이익률'] = kosdaq['영업손익']/kosdaq['매출액']*100
kosdaq['총자본영업이익률'] = kosdaq['영업손익']/kosdaq['자산']*100
kosdaq['자기자본영업이익률'] = kosdaq['영업손익']/kosdaq['자본']*100
kosdaq['매출원가 대 매출액 비율'] = kosdaq['매출원가']/kosdaq['매출액']*100
kosdaq['영업비용 대 영업수익 비율'] = kosdaq['판관비']/kosdaq['영업손익']*100
kosdaq['총비용'] = kosdaq['매출원가']+kosdaq['판관비']+kosdaq['이자비용']+kosdaq['감가상각비']
kosdaq['감가상각비 대 총비용비율'] = kosdaq['감가상각비']/kosdaq['총비용']*100

In [180]:
kosdaq.columns[kosdaq.isnull().sum() == 1403] # 첫년도 비율 구할 수 없어서 결측치 존재 (M_Score에서도 계산 못하기 때문에 OK)

Index(['총자본증가율', '유형자산증가율', '비유동자산증가율', '유동자산증가율', '매출액증가율', '자기자본회전률',
       '매입채무회전률', '매입채무회전기간', '유동자산회전률', '당좌자산회전률', '재고자산회전률', '재고자산회전기간',
       '매출채권회전률', '매출채권회전기간', '비유동자산회전률', '유형자산회전율', '순운전자본회전률', '1회전기간'],
      dtype='object')

In [181]:
# 재고자산이 0이기 때문에 결측치 발생
kosdaq[kosdaq['재고자산증가율'].isna()]
kosdaq['재고자산증가율'] = kosdaq['재고자산증가율'].fillna(0.0)

In [182]:
# 매입채무, 매입채무 및 기타유동채무가 모두 0이기 때문에 NaN -> 0.0으로 변경
kosdaq[kosdaq['매입채무 대 재고자산비율'].isna()][['회사명', '거래소코드', '회계년도', '재고자산', '매입채무 대 재고자산비율', '매입채무 및 기타유동채무']]
kosdaq['매입채무 대 재고자산비율'] = kosdaq['매입채무 대 재고자산비율'].fillna(0.0)

In [183]:
# OCF, 장기차입금, 단기차입금, 사채가 모두 0이기 때문에 NaN -> 0.0으로 변경
kosdaq[kosdaq['CASH FLOW 대 차입금비율'].isna()][['거래소코드', '회사명', '회계년도', 'OCF', '장기차입금', '단기차입금', '사채']]
kosdaq['CASH FLOW 대 차입금비율'] = kosdaq['CASH FLOW 대 차입금비율'].fillna(0.0)

In [184]:
# 장기차입금, 단기차입금, 사채가 모두 0이기 때문에 NaN -> 0.0으로 변경
kosdaq[kosdaq['단기차입금 대 총차입금비율'].isna()][['거래소코드', '회사명', '회계년도', '장기차입금', '단기차입금', '사채']]
kosdaq['단기차입금 대 총차입금비율'] = kosdaq['단기차입금 대 총차입금비율'].fillna(0.0)

In [185]:
# 매출액, OCF가 모두 0이기 때문에 NaN -> 모두 0으로 처리
kosdaq[kosdaq['CASH FLOW 대 매출액비율'].isna()][['거래소코드', '회사명', '회계년도', 'OCF', '매출액']]
kosdaq['CASH FLOW 대 매출액비율'] = kosdaq['CASH FLOW 대 매출액비율'].fillna(0.0)

In [186]:
# kosdaq.loc[kosdaq['종업원수'].isna(), '종업원수'] = kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['종업원수'].shift(-1)

In [187]:
kosdaq.loc[(kosdaq['종업원수']==0.0) | (kosdaq['종업원수'].isna()), '종업원수'] = kosdaq.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['종업원수'].shift(-1)
kosdaq['종업원수'].fillna(0.0, inplace=True)

In [188]:
kosdaq.isnull().sum().unique()

array([   0, 1403], dtype=int64)

코스피

In [190]:
kospi['총자본증가율'] = (kospi['자산'] - kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['자산'].shift(1)) / kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['자산'].shift(1) * 100
kospi['유형자산증가율'] = (kospi['유형자산'] - kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유형자산'].shift(1)) / kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유형자산'].shift(1) * 100
kospi['비유동자산증가율'] = (kospi['비유동자산'] - kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1)) / kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1) * 100
kospi['유동자산증가율'] = (kospi['유동자산'] - kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유동자산'].shift(1)) / kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유동자산'].shift(1) * 100
kospi['매출액증가율'] = (kospi['매출액'] - kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매출액'].shift(1)) / kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매출액'].shift(1) * 100
kospi['매출액총이익률'] = kospi['매출총이익']/kospi['매출액']*100
kospi['유동비율'] = kospi['유동자산']/kospi['자산']*100
kospi['유동자산 대 비유동자산비율'] = kospi['유동자산']/kospi['비유동자산']*100
kospi['비유동자산구성비율'] = kospi['비유동자산']/kospi['자산']*100
kospi['자기자본구성비율'] = kospi['자본']/kospi['자산']*100
kospi['타인자본구성비율'] = kospi['부채']/kospi['자산']*100
kospi['비유동비율'] = kospi['비유동자산']/kospi['자본']*100
kospi['비유동장기적합률'] = kospi['비유동자산']/(kospi['부채']-kospi['유동부채']+kospi['자본'])*100
kospi['유동비율'] = kospi['유동자산']/kospi['유동부채']*100
kospi['현금비율'] = kospi['현금및현금성자산']/kospi['자산']*100
kospi['매출채권비율'] = kospi['매출채권 및 기타유동채권']/kospi['자산']*100
kospi['순운전자본'] = kospi['유동자산']-kospi['유동부채']
kospi['부채비율'] = kospi['부채']/kospi['자본']*100
kospi['유동부채비율'] = kospi['유동부채']/kospi['자본']*100
kospi['비유동부채비율'] = (kospi['부채']-kospi['유동부채'])/kospi['자본']*100
kospi['비유동부채 대 순운전자본비율'] = (kospi['부채']-kospi['유동부채'])/kospi['순운전자본']*100
kospi['순운전자본비율'] = kospi['순운전자본']/kospi['자산']*100
kospi['차입금의존도'] = (kospi['장기차입금']+kospi['단기차입금']+kospi['사채'])/kospi['자산']*100
kospi['차입금비율'] = (kospi['장기차입금']+kospi['단기차입금']+kospi['사채'])/kospi['자본']*100
kospi['이자보상배율(이자비용)'] = kospi['영업손익']/kospi['이자비용']
kospi['CASH FLOW 대 부채비율'] = kospi['OCF']/kospi['부채']*100
kospi['CASH FLOW 대 차입금비율'] = kospi['OCF']/(kospi['장기차입금']+kospi['단기차입금']+kospi['사채'])*100
kospi['CASH FLOW 대 매출액비율'] = kospi['OCF']/kospi['매출액']*100
kospi['총자본회전률'] = kospi['매출액']/kospi['자산']*100
kospi['자기자본회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['자본'].shift(1) + kospi['자본'])*100
kospi['매입채무회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매입채무 및 기타유동채무'].shift(1) + kospi['매입채무 및 기타유동채무'])*100
kospi['매입채무회전기간'] = 365/kospi['매입채무회전률']*100
kospi['유동자산회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유동자산'].shift(1) + kospi['유동자산'])*100
kospi['당좌자산'] = kospi['유동자산']-kospi['재고자산']
kospi['당좌자산회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['당좌자산'].shift(1) + kospi['당좌자산'])*100
kospi['재고자산회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['재고자산'].shift(1) + kospi['재고자산'])*100
kospi['재고자산회전기간'] = 365/kospi['재고자산회전률']*100
kospi['매출채권회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['매출채권 및 기타유동채권'].shift(1) + kospi['매출채권 및 기타유동채권'])*100
kospi['매출채권회전기간'] = 365/kospi['매출채권회전률']*100
kospi['비유동자산회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1) + kospi['비유동자산'])*100
kospi['유형자산회전율'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['유형자산'].shift(1) + kospi['유형자산'])*100
kospi['비유동자산회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['비유동자산'].shift(1) + kospi['비유동자산'])*100
kospi['순운전자본회전률'] = kospi['매출액']/(kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['순운전자본'].shift(1) + kospi['순운전자본'])*100
kospi['1회전기간'] = kospi['매출채권회전기간'] + kospi['재고자산회전기간'] - kospi['매입채무회전기간']
kospi['당좌자산구성비율'] = kospi['당좌자산']/kospi['자산']*100
kospi['당좌비율'] = kospi['당좌자산']/kospi['유동부채']*100
kospi['재고자산 대 순운전자본비율'] = kospi['재고자산']/kospi['순운전자본']*100
kospi['재고자산증가율'] = (kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['재고자산'].shift(1) + kospi['재고자산'])/kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['재고자산'].shift(1)*100
kospi['매출채권 대 매입채무비율'] = kospi['매출채권 및 기타유동채권']/kospi['매입채무 및 기타유동채무']*100
kospi['매입채무 대 재고자산비율'] = kospi['매입채무 및 기타유동채무']/kospi['재고자산']*100
kospi['단기차입금 대 총차입금비율'] = kospi['단기차입금']/(kospi['단기차입금']+kospi['장기차입금']+kospi['사채'])*100
kospi['매출액영업이익률'] = kospi['영업손익']/kospi['매출액']*100
kospi['총자본영업이익률'] = kospi['영업손익']/kospi['자산']*100
kospi['자기자본영업이익률'] = kospi['영업손익']/kospi['자본']*100
kospi['매출원가 대 매출액 비율'] = kospi['매출원가']/kospi['매출액']*100
kospi['영업비용 대 영업수익 비율'] = kospi['판관비']/kospi['영업손익']*100
kospi['총비용'] = kospi['매출원가']+kospi['판관비']+kospi['이자비용']+kospi['감가상각비']
kospi['감가상각비 대 총비용비율'] = kospi['감가상각비']/kospi['총비용']*100

In [191]:
kospi.columns[kospi.isnull().sum() == 757] # 첫년도 비율 구할 수 없어서 결측치 존재 (M_Score에서도 계산 못하기 때문에 OK)

Index(['총자본증가율', '유형자산증가율', '비유동자산증가율', '유동자산증가율', '매출액증가율', '자기자본회전률',
       '매입채무회전률', '매입채무회전기간', '유동자산회전률', '당좌자산회전률', '재고자산회전률', '재고자산회전기간',
       '매출채권회전률', '매출채권회전기간', '비유동자산회전률', '유형자산회전율', '순운전자본회전률', '1회전기간'],
      dtype='object')

In [192]:
# 재고자산이 0이기 때문에 결측치 발생
kospi[kospi['재고자산증가율'].isna()]
kospi['재고자산증가율'] = kospi['재고자산증가율'].fillna(0.0)

In [193]:
# 장기차입금, 단기차입금, 사채가 모두 0이기 때문에 NaN -> 0.0으로 변경
kospi[kospi['단기차입금 대 총차입금비율'].isna()][['거래소코드', '회사명', '회계년도', '장기차입금', '단기차입금', '사채']]
kospi['단기차입금 대 총차입금비율'] = kospi['단기차입금 대 총차입금비율'].fillna(0.0)

In [194]:
kospi.loc[(kospi['종업원수']==0.0) | (kospi['종업원수'].isna()), '종업원수'] = kospi.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['종업원수'].shift(-1)
kospi['종업원수'].fillna(0.0, inplace=True)

In [195]:
# OCF, 장기차입금, 단기차입금, 사채가 모두 0이기 때문에 NaN -> 0.0으로 변경
kospi[kospi['CASH FLOW 대 차입금비율'].isna()][['거래소코드', '회사명', '회계년도', 'OCF', '장기차입금', '단기차입금', '사채']]
kospi['CASH FLOW 대 차입금비율'] = kospi['CASH FLOW 대 차입금비율'].fillna(0.0)

In [196]:
# 매출액, 매출총이익이 0이기 때문에 NaN -> 0.0으로 변경
kospi[kospi['매출액총이익률'].isna()][['회사명', '거래소코드', '회계년도', '매출액', '매출총이익']]
kospi['매출액총이익률'] = kospi['매출액총이익률'].fillna(0.0)

In [197]:
# 매출액, 매출원가가 0이기 때문에 NaN -> 0.0으로 변경
kospi[kospi['매출원가 대 매출액 비율'].isna()][['회사명', '거래소코드', '회계년도', '매출원가', '매출액']]
kospi['매출원가 대 매출액 비율'] = kospi['매출원가 대 매출액 비율'].fillna(0.0)

In [198]:
kospi.isnull().sum().unique()

array([  0,   9, 757], dtype=int64)

#### Beneish M-Score

M-score = −4.84 + 0.92 * DSRI + 0.528 * GMI + 0.404 * AQI + 0.892 * SGI + 0.115 * DEPI −0.172 * SGAI + 4.679 * TATA − 0.327 * LVGI

In [214]:
def m_score(df):
    tmp_df = df.copy()

    tmp_df['DSRI_y'] = tmp_df['매출채권 및 기타유동채권'] / tmp_df['매출액']
    tmp_df['GMI_y'] = (tmp_df['매출액'] - tmp_df['매출원가']) / tmp_df['매출액']
    tmp_df['AQI_y'] = 1 - ( (tmp_df['유동자산'] + tmp_df['유형자산']) / tmp_df['자산'] )
    tmp_df['SGI_y'] = tmp_df['매출액']
    tmp_df['DEPI_y'] = tmp_df['감가상각비'] / (tmp_df['유형자산'] + tmp_df['감가상각비'])
    tmp_df['SGAI_y'] = tmp_df['판관비'] / tmp_df['매출액']
    tmp_df['LVGI_y'] = (tmp_df['유동부채'] + tmp_df['비유동부채 ']) / tmp_df['자산']

    tmp_df.loc[tmp_df['매출액']==0.0, 'DSRI_y'] = 0.0
    tmp_df.loc[tmp_df['매출액']==0.0, 'GMI_y'] = 0.0
    tmp_df.loc[tmp_df['매출액']==0.0, 'SGAI_y'] = 0.0

    tmp_df['DSRI'] = tmp_df['DSRI_y']/tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['DSRI_y'].shift(1)
    tmp_df['GMI'] = tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['GMI_y'].shift(1)/tmp_df['GMI_y']
    tmp_df['AQI'] = tmp_df['AQI_y']/tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['AQI_y'].shift(1)
    tmp_df['SGI'] = tmp_df['SGI_y']/tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['SGI_y'].shift(1)
    tmp_df['DEPI'] = tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['DEPI_y'].shift(1)/tmp_df['DEPI_y']
    tmp_df['SGAI'] = tmp_df['SGAI_y']/tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['SGAI_y'].shift(1)
    tmp_df['LVGI'] = tmp_df['LVGI_y']/tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['LVGI_y'].shift(1)
    
    tmp_df['TATA'] = (tmp_df['당기순이익'] - tmp_df['OCF'] - tmp_df['ICF']) / tmp_df['자산']

    tmp_df.loc[tmp_df['DEPI'].isna(), 'DEPI'] = 0.0  # 감가상각비 0인 row 때문에
    tmp_df.loc[tmp_df['DSRI']==np.inf, 'DSRI'] = 0.0
    tmp_df.loc[tmp_df['GMI']==np.inf, 'GMI'] = 0.0
    tmp_df.loc[tmp_df['SGI']==np.inf, 'SGI'] = 0.0
    tmp_df.loc[tmp_df['SGAI']==np.inf, 'SGAI'] = 0.0

    tmp_df['M_Score'] = -4.84 + 0.92 * tmp_df['DSRI'] + 0.528 * tmp_df['GMI'] + 0.404 * tmp_df['AQI'] + 0.892 * tmp_df['SGI'] + 0.115 * tmp_df['DEPI'] - 0.172 * tmp_df['SGAI'] + 4.679 * tmp_df['TATA'] - 0.327 * tmp_df['LVGI']
    #df['M_Score'] = tmp_df['M_Score']

    tmp_df['회계년도y_1']= (tmp_df.sort_values(by='회계년도', ascending=True).groupby('거래소코드')['회계년도'].shift(1)).fillna(-50).astype('int64')
    tmp_df['회계년도 차'] = tmp_df['회계년도'] - tmp_df['회계년도y_1']
    tmp_df.drop(tmp_df[tmp_df['회계년도 차']!=1].index, axis=0, inplace=True)

    tmp_df.drop(['DSRI', 'GMI', 'AQI', 'SGI', 'DEPI', 'SGAI', 'LVGI', 'TATA', 'DSRI_y', 'GMI_y', 'AQI_y', 'SGI_y', 'DEPI_y', 'SGAI_y', 'LVGI_y', '회계년도y_1', '회계년도 차'], axis=1, inplace=True)
    tmp_df.drop(tmp_df[tmp_df['M_Score']==np.inf].index, axis=0, inplace=True) # 유형자산 == 비유동자산 -> AQI의 분모가 0이라서 M_Score inf
    return tmp_df

In [215]:
kosdaq_all = m_score(kosdaq)
kosdaq_all[kosdaq_all['M_Score'].isna()==False].sort_values('M_Score')

,거래소코드,회사명,회계년도,유동자산,비유동자산,유동부채,비유동부채,자본금,자본잉여금,기타포괄손익누계액,...,순운전자본,당좌자산,매출액영업이익률,총자본영업이익률,자기자본영업이익률,매출원가 대 매출액 비율,영업비용 대 영업수익 비율,총비용,감가상각비 대 총비용비율,M_Score
6559,222110,(주)팬젠,2014,4029.00000,12844.00000,5554.00000,3107.00000,1443.00000,12674.00000,0.00000,...,-1525.00000,4029.00000,-59.44118,-11.97772,-24.61033,100.02941,-99.95052,7353.87000,13.25656,-266.52525
9005,244460,올리패스(주),2016,15425.00000,20552.00000,14116.00000,3174.00000,5749.00000,26079.00000,0.00000,...,1309.00000,15425.00000,-433100.00000,-24.07716,-46.35556,0.00000,-100.02309,11525.07000,16.47773,-118.63073
1200,197210,(주)리드,2019,27454.00000,3592.00000,61994.00000,615.00000,12530.00000,93036.00000,0.00000,...,-34540.00000,26585.00000,-35.94260,-21.29743,20.94924,100.44031,-98.79008,31414.51000,2.72011,-44.09348
4465,091270,(주)유디피,2017,1186.00000,443.00000,21435.00000,812.00000,3669.00000,0.00000,0.00000,...,-20249.00000,1022.00000,-24.35089,-327.01044,25.83539,77.82044,-191.06439,28151.93000,0.62848,-41.60529
8439,900120,씨케이에이치푸드앤헬스리미티드,2018,440058.00000,120771.00000,37399.00000,0.00000,6415.00000,145290.00000,0.00000,...,402659.00000,434841.00000,-52.66067,-13.63999,-14.61456,100.50735,-99.03656,223470.91000,0.76516,-38.80356
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9491,033050,제이엠아이(주),2013,90484.00000,65238.00000,102411.00000,551.00000,32579.00000,13364.00000,1041.00000,...,-11927.00000,72808.00000,-4.69366,-7.46844,-22.04321,100.04682,-99.01118,271129.64000,3.69736,57.46702
7784,046400,디브이에스코리아(주),2013,10993.00000,3783.00000,5116.00000,340.00000,9267.00000,34026.00000,0.00000,...,5877.00000,10108.00000,23.26992,5.71197,9.05677,99.86215,-99.40758,3280.07000,8.38610,61.40718
4325,039790,(주)위노바,2013,30850.00000,19286.00000,21018.00000,593.00000,27150.00000,19103.00000,-87.00000,...,9832.00000,23370.00000,-39.91049,-19.92221,-35.01613,99.83217,-100.43052,37878.68000,3.79813,65.08359
2041,011370,(주)서한,2016,342451.00000,70218.00000,173561.00000,26696.00000,50447.00000,5743.00000,-431.00000,...,168890.00000,215460.00000,17.91108,21.68203,42.12333,78.55514,19.72953,413150.20100,0.00005,78.67822


In [216]:
kospi_all = m_score(kospi)
kospi_all[kospi_all['M_Score'].isna()==False].sort_values('M_Score')

,거래소코드,회사명,회계년도,유동자산,비유동자산,유동부채,비유동부채,자본금,자본잉여금,기타포괄손익누계액,...,순운전자본,당좌자산,매출액영업이익률,총자본영업이익률,자기자본영업이익률,매출원가 대 매출액 비율,영업비용 대 영업수익 비율,총비용,감가상각비 대 총비용비율,M_Score
3938,001470,삼부토건(주),2015,1316393.00000,469732.00000,1956496.00000,89257.00000,44013.00000,28316.00000,651062.00000,...,-640103.00000,1260626.00000,-11.98267,-3.54253,24.37093,100.02102,-99.82615,698570.71000,1.79319,-127.72585
2433,019490,(주)하이트론씨스템즈,2017,34265.00000,23617.00000,20931.00000,4272.00000,13827.00000,34677.00000,-542.00000,...,13334.00000,18867.00000,-30.36731,-19.71079,-34.91233,100.06388,-99.78964,50540.32000,2.65594,-44.12577
2073,006380,(주)카프로,2012,367148.00000,367488.00000,148300.00000,129330.00000,20000.00000,32384.00000,52020.00000,...,218848.00000,296558.00000,-2.51127,-3.27019,-5.25682,100.30022,-88.04529,1012678.06000,2.80514,-37.56513
3518,030720,동원수산(주),2013,72891.00000,48564.00000,37213.00000,40262.00000,18744.00000,13082.00000,544.00000,...,35678.00000,35207.00000,-9.15115,-8.81232,-24.33606,100.23684,-97.41194,132588.20000,2.27335,-25.05098
1306,010580,(주)에스엠벡셀,2013,29430.00000,49192.00000,43078.00000,17931.00000,13856.00000,3431.00000,-4.00000,...,-13648.00000,21444.00000,-2.36289,-2.74100,-12.23598,99.89255,-104.54756,97351.79000,2.81946,-22.43221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3895,030790,비케이탑스(주),2019,61848.00000,33655.00000,52786.00000,3540.00000,74160.00000,108933.00000,-44562.00000,...,9062.00000,61086.00000,-25.22492,-14.70859,-35.85614,100.03412,-99.86474,84913.87000,1.42600,83.18750
1748,000700,(주)유수홀딩스,2013,2065316.00000,8763525.00000,6196495.00000,3912440.00000,218954.00000,273973.00000,-13083.00000,...,-4131179.00000,1742162.00000,-1.99406,-1.90764,-28.69472,98.01355,-199.61806,11436007.02000,3.97665,93.43629
1107,134790,(주)시디즈,2014,20619.00000,2398.00000,831.00000,377.00000,1000.00000,20165.00000,0.00000,...,19788.00000,19352.00000,-4.52816,-2.12017,-2.23761,99.94433,-101.22951,11563.27000,2.57946,98.58057
4641,326030,에스케이바이오팜(주),2019,98681.00000,40392.00000,145641.00000,6458.00000,32500.00000,446040.00000,-355.00000,...,-46960.00000,91948.00000,-63.99493,-56.99093,608.51440,0.41501,-255.61387,211158.92000,3.07869,99.29646


#### 이상치 처리

코스닥

In [217]:
# inf 값 컬럼별 갯수 확인
for i in range(len(kosdaq_all.columns)):
    col = kosdaq_all.columns[i]
    cnt = len( kosdaq_all[(kosdaq_all[col]==np.inf) | (kosdaq_all[col]==-np.inf)] )
    if cnt > 0 :
        print(col, cnt)

유형자산증가율 1
재고자산증가율 101
매출채권 대 매입채무비율 8
매입채무 대 재고자산비율 629
이자보상배율(이자비용) 698
CASH FLOW 대 차입금비율 1506
매입채무회전률 7
재고자산회전률 560


In [218]:
kosdaq_all.drop(['유형자산증가율', '재고자산증가율', '매출채권 대 매입채무비율', '매입채무 대 재고자산비율', '매입채무회전률', '재고자산회전률'], axis=1, inplace=True)

In [219]:
# 정상기업에서 inf, -inf 값은 삭제
# kosdaq_all.drop(kosdaq_all[((kosdaq_all['이자보상배율(이자비용)'] == np.inf) | (kosdaq_all['이자보상배율(이자비용)'] == -np.inf)) & (kosdaq_all['target']==0)].index, axis=0, inplace=True)

# -inf 값은 0으로 대체
kosdaq_all.loc[kosdaq_all['이자보상배율(이자비용)'] == -np.inf, '이자보상배율(이자비용)'] = 0

# inf 값은 이자보상배율 중 최대값으로 대체
kosdaq_all.loc[kosdaq_all['이자보상배율(이자비용)'] == np.inf, '이자보상배율(이자비용)'] = np.nan
kosdaq_all['이자보상배율(이자비용)'].fillna(kosdaq_all['이자보상배율(이자비용)'].max(), inplace=True)

In [220]:
# 정상기업에서 inf, -inf 값은 삭제
# kosdaq_all.drop(kosdaq_all[((kosdaq_all['CASH FLOW 대 차입금비율'] == np.inf) | (kosdaq_all['CASH FLOW 대 차입금비율'] == -np.inf)) & (kosdaq_all['target']==0)].index, axis=0, inplace=True)

# -inf 값은 0으로 대체
kosdaq_all.loc[kosdaq_all['CASH FLOW 대 차입금비율'] == -np.inf, 'CASH FLOW 대 차입금비율'] = 0

# inf 값은 이자보상배율 중 최대값으로 대체
kosdaq_all.loc[kosdaq_all['CASH FLOW 대 차입금비율'] == np.inf, 'CASH FLOW 대 차입금비율'] = np.nan
kosdaq_all['CASH FLOW 대 차입금비율'].fillna(kosdaq_all['CASH FLOW 대 차입금비율'].max(), inplace=True)

코스피

In [221]:
for i in range(len(kospi_all.columns)):
    col = kospi_all.columns[i]
    cnt = len( kospi_all[(kospi_all[col]==np.inf) | (kospi_all[col]==-np.inf)] )
    if cnt > 0 :
        print(col, cnt)

재고자산증가율 20
매출액증가율 2
매입채무 대 재고자산비율 131
이자보상배율(이자비용) 215
CASH FLOW 대 차입금비율 459
재고자산회전률 112


In [222]:
# 컬럼 drop
kospi_all.drop(['재고자산증가율', '매입채무 대 재고자산비율', '재고자산회전률'], axis=1, inplace=True)

# row drop
kospi_all.drop(kospi_all[(kospi_all['매출액증가율']==np.inf) | (kospi_all['매출액증가율']==-np.inf)].index, axis=0, inplace=True)
kospi_all.drop(kospi_all[(kospi_all['CASH FLOW 대 매출액비율']==np.inf) | (kospi_all['CASH FLOW 대 매출액비율']==-np.inf)].index, axis=0, inplace=True)

In [223]:
# 정상기업에서 inf, -inf 값은 삭제
# kospi_all.drop(kospi_all[((kospi_all['이자보상배율(이자비용)'] == np.inf) | (kospi_all['이자보상배율(이자비용)'] == -np.inf)) & (kospi_all['target']==0)].index, axis=0, inplace=True)

# -inf 값은 0으로 대체
kospi_all.loc[kospi_all['이자보상배율(이자비용)'] == -np.inf, '이자보상배율(이자비용)'] = 0
# inf 값은 이자보상배율 중 최대값으로 대체
kospi_all.loc[kospi_all['이자보상배율(이자비용)'] == np.inf, '이자보상배율(이자비용)'] = np.nan
kospi_all['이자보상배율(이자비용)'].fillna(kospi_all['이자보상배율(이자비용)'].max(), inplace=True)

In [224]:
# 정상기업에서 inf, -inf 값은 삭제
# kospi_all.drop(kospi_all[((kospi_all['CASH FLOW 대 차입금비율'] == np.inf) | (kospi_all['CASH FLOW 대 차입금비율'] == -np.inf)) & (kospi_all['target']==0)].index, axis=0, inplace=True)

# -inf 값은 0으로 대체
kospi_all.loc[kospi_all['CASH FLOW 대 차입금비율'] == -np.inf, 'CASH FLOW 대 차입금비율'] = 0
# inf 값은 이자보상배율 중 최대값으로 대체
kospi_all.loc[kospi_all['CASH FLOW 대 차입금비율'] == np.inf, 'CASH FLOW 대 차입금비율'] = np.nan
kospi_all['CASH FLOW 대 차입금비율'].fillna(kospi_all['CASH FLOW 대 차입금비율'].max(), inplace=True)

#### 컬럼 drop

절대적인 수치로는 판단하기 어려운 변수들이기 때문에 drop

해당 변수들로 재무비율 계산하여 파생변수 생성

-> 파생변수를 가지고 부도 예측을 하는게 더 합리적이라고 판단하였음

코스닥

In [225]:
log_list_daq = ['유동자산', '비유동자산', '유동부채', '비유동부채 ', '자본금',
                '자본잉여금', '기타포괄손익누계액', '이익잉여금', '매출액', '매출원가', '매출총이익', '판관비',
                '영업손익', '금융수익', '기타이익', '기타손실', '법인세비용', '당기순이익', '총포괄손익',
                'ICF', 'FCF', '매출채권 및 기타유동채권', '현금및현금성자산', '유형자산', '자산',
                '부채', '자본', '감가상각비', '재고자산', '매입채무 및 기타유동채무', '단기차입금', '장기차입금',
                '사채', '이자비용', '순운전자본', '당좌자산', '총비용']
kosdaq_all.drop(log_list_daq, axis=1, inplace=True)

코스피

In [226]:
log_list_pi = ['유동자산', '비유동자산', '유동부채', '비유동부채 ', '자본금', '자본잉여금', '기타포괄손익누계액', 
               '이익잉여금', '매출액', '매출원가', '매출총이익', '판관비', '영업손익', '금융수익', 
               '기타이익', '기타손실', '법인세비용', '당기순이익', '총포괄손익', 'ICF', 'FCF',
               '매출채권 및 기타유동채권', '현금및현금성자산', '유형자산', '자산', '부채', '자본', '감가상각비',
               '재고자산', '매입채무 및 기타유동채무', '단기차입금', '장기차입금', '사채', '이자비용', '순운전자본',
               '당좌자산', '총비용']
kospi_all.drop(log_list_pi, axis=1, inplace=True)

#### 윈저라이징

In [227]:
# 시각화 : 윈저라이진 시행 전/후 비교를 위해 시각화시키고자 함수 생성

def outliers_visual(data):
    plt.figure(figsize=(15, 40))
    i = 0
    for col in cont_vars:
        i += 1
        plt.subplot(50, 4, i)
        plt.boxplot(data[col])
        plt.title('{} boxplot'.format(col))
        i += 1
        plt.subplot(50, 4, i)
        plt.hist(data[col])
        plt.title('{} histogram'.format(col))
    plt.show()

In [228]:
# 이상치 변환

def test_wins(df, col, lower_limit=0, upper_limit=0, show_plot=True):
    wins_data = winsorize(df[col], limits=(lower_limit, upper_limit))
    # tmp_df[col] = wins_data
    if show_plot == True:
        plt.figure(figsize=(19,5))
        plt.subplot(121)
        plt.hist(df[col])
        plt.title('original {}'.format(col))
        plt.subplot(122)
        plt.hist(wins_data)
        plt.title('wins=({},{}) {}'.format(lower_limit, upper_limit, col))
        plt.show()
    return wins_data

코스피

In [232]:
kospi_all_wins = kospi_all.copy()

# 각 컬럼들을 반복적으로 윈저라이징 시키기 위해 for 문 생성
# target 은 윈저라이징 시키면 안 되니까 pass 하기위해 continue 사용

kospi_col = kospi_all_wins.columns[3:]
for i in range(len(kospi_col)):
    col = kospi_col.values[i]
    if col == 'target':
        continue
    kospi_all_wins[col] = test_wins(kospi_all_wins, col, lower_limit=0.05, upper_limit=0.05, show_plot=False)

코스닥

In [233]:
kosdaq_all_wins = kosdaq_all.copy()

kosdaq_col = kosdaq_all_wins.columns[3:]
for i in range(len(kosdaq_col)):
    col = kosdaq_col.values[i]
    if col == 'target':
        continue
    kosdaq_all_wins[col] = test_wins(kosdaq_all_wins, col, lower_limit=0.05, upper_limit=0.05, show_plot=False)

#### Dataset 정리

In [234]:
def year_drop(df_all):

    df_all.drop(df_all[(df_all['회계년도+1']==2020)].index, axis=0, inplace=True)
    df_all.drop('회계년도+1', axis=1, inplace=True)

    bankruptcy_df = df_all[df_all['target']==1]['거래소코드'].unique()
    
    for i in range(len(bankruptcy_df)):
        tmp_df = df_all[df_all['거래소코드']==bankruptcy_df[i]]
        min_year = tmp_df[tmp_df['target']==1]['회계년도'].min()
        df_all.drop(tmp_df[tmp_df['회계년도'] > min_year].index, axis=0, inplace=True)

In [97]:
year_drop(kosdaq_all)
year_drop(kospi_all)
year_drop(kosdaq_all_wins)
year_drop(kospi_all_wins)

#### 회계 부정 dataset

##### 코스닥

In [98]:
kosdaq_all['M_Score'] = np.where(kosdaq_all['M_Score'] >= -2.2, 1, 0)
kosdaq_all_wins['M_Score'] = np.where(kosdaq_all_wins['M_Score'] >= -2.2, 1, 0)

In [99]:
kosdaq_m_score = kosdaq_all.groupby('거래소코드')['M_Score'].sum()/kosdaq_all.groupby('거래소코드')['M_Score'].count()

for i in range(len(kosdaq_m_score)):
    if kosdaq_m_score.values[i] == 0 :
        kosdaq_all.loc[kosdaq_all['거래소코드']==kosdaq_m_score.index[i], 'M_Score'] = 0
    else:
        kosdaq_all.loc[kosdaq_all['거래소코드']==kosdaq_m_score.index[i], 'M_Score'] = 1

In [100]:
kosdaq_m_score = kosdaq_all_wins.groupby('거래소코드')['M_Score'].sum()/kosdaq_all_wins.groupby('거래소코드')['M_Score'].count()

for i in range(len(kosdaq_m_score)):
    if kosdaq_m_score.values[i] == 0 :
        kosdaq_all_wins.loc[kosdaq_all_wins['거래소코드']==kosdaq_m_score.index[i], 'M_Score'] = 0
    else:
        kosdaq_all_wins.loc[kosdaq_all_wins['거래소코드']==kosdaq_m_score.index[i], 'M_Score'] = 1

In [235]:
print('<코스닥 전체>\n')
print('정상 기업 수 :',len(kosdaq_all[(kosdaq_all['target']==0)]['거래소코드'].unique()))
print('정상 데이터 수 :',len(kosdaq_all[(kosdaq_all['target']==0)]))
print()
print('부실 기업 수 :',len(kosdaq_all[(kosdaq_all['target']==1)]['거래소코드'].unique()))
print('부실 데이터 수 :',len(kosdaq_all[(kosdaq_all['target']==1)]))

<코스닥 전체>

정상 기업 수 : 1400
정상 데이터 수 : 7796

부실 기업 수 : 617
부실 데이터 수 : 1041


In [236]:
print('<회계 부정>\n')
print('부정 기업 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==1)]['거래소코드'].unique()))
print('부정 데이터 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==1)]))
print()
print('정상 기업 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==1)&(kosdaq_all['target']==0)]['거래소코드'].unique()))
print('정상 데이터 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==1)&(kosdaq_all['target']==0)]))
print()
print('부실 기업 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==1)&(kosdaq_all['target']==1)]['거래소코드'].unique()))
print('부실 데이터 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==1)&(kosdaq_all['target']==1)]))

<회계 부정>

부정 기업 수 : 0
부정 데이터 수 : 0

정상 기업 수 : 0
정상 데이터 수 : 0

부실 기업 수 : 0
부실 데이터 수 : 0


In [237]:
print('<회계 건전>\n')
print('건전 기업 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==0)]['거래소코드'].unique()))
print('건전 데이터 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==0)]))
print()
print('정상 기업 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==0)&(kosdaq_all['target']==0)]['거래소코드'].unique()))
print('정상 데이터 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==0)&(kosdaq_all['target']==0)]))
print()
print('부실 기업 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==0)&(kosdaq_all['target']==1)]['거래소코드'].unique()))
print('부실 데이터 수 :',len(kosdaq_all[(kosdaq_all['M_Score']==0)&(kosdaq_all['target']==1)]))

<회계 건전>

건전 기업 수 : 0
건전 데이터 수 : 0

정상 기업 수 : 0
정상 데이터 수 : 0

부실 기업 수 : 0
부실 데이터 수 : 0


In [238]:
kosdaq_good = kosdaq_all[(kosdaq_all['M_Score'] == 0)]  # M1 정상
kosdaq_bad = kosdaq_all[(kosdaq_all['M_Score'] == 1)]  # M1 부정

kosdaq_good_wins = kosdaq_all_wins[(kosdaq_all_wins['M_Score'] == 0)]  # M1 정상
kosdaq_bad_wins = kosdaq_all_wins[(kosdaq_all_wins['M_Score'] == 1)]  # M1 부정

In [239]:
kosdaq_all.drop('M_Score', axis=1, inplace=True)
kosdaq_good.drop('M_Score', axis=1, inplace=True)
kosdaq_bad.drop('M_Score', axis=1, inplace=True)
kosdaq_all_wins.drop('M_Score', axis=1, inplace=True)
kosdaq_good_wins.drop('M_Score', axis=1, inplace=True)
kosdaq_bad_wins.drop('M_Score', axis=1, inplace=True)

##### 코스피

In [240]:
kospi_all['M_Score'] = np.where(kospi_all['M_Score'] >= -2.2, 1, 0)
kospi_all_wins['M_Score'] = np.where(kospi_all_wins['M_Score'] >= -2.2, 1, 0)

In [241]:
kospi_m_score = kospi_all.groupby('거래소코드')['M_Score'].sum()/kospi_all.groupby('거래소코드')['M_Score'].count()
for i in range(len(kospi_m_score)):
    if kospi_m_score.values[i] == 0 :
        kospi_all.loc[kospi_all['거래소코드']==kospi_m_score.index[i], 'M_Score'] = 0
    else:
        kospi_all.loc[kospi_all['거래소코드']==kospi_m_score.index[i], 'M_Score'] = 1

In [242]:
kospi_m_score = kospi_all_wins.groupby('거래소코드')['M_Score'].sum()/kospi_all_wins.groupby('거래소코드')['M_Score'].count()

for i in range(len(kospi_m_score)):
    if kospi_m_score.values[i] == 0 :
        kospi_all_wins.loc[kospi_all_wins['거래소코드']==kospi_m_score.index[i], 'M_Score'] = 0
    else:
        kospi_all_wins.loc[kospi_all_wins['거래소코드']==kospi_m_score.index[i], 'M_Score'] = 1

In [243]:
print('<코스피 전체>\n')
print('정상 기업 수 :',len(kospi_all[(kospi_all['target']==0)]['거래소코드'].unique()))
print('정상 데이터 수 :',len(kospi_all[(kospi_all['target']==0)]))
print()
print('부실 기업 수 :',len(kospi_all[(kospi_all['target']==1)]['거래소코드'].unique()))
print('부실 데이터 수 :',len(kospi_all[(kospi_all['target']==1)]))

<코스피 전체>

정상 기업 수 : 757
정상 데이터 수 : 5026

부실 기업 수 : 273
부실 데이터 수 : 501


In [244]:
print('<회계 부정>\n')
print('부정 기업 수 :',len(kospi_all[(kospi_all['M_Score']==1)]['거래소코드'].unique()))
print('부정 데이터 수 :',len(kospi_all[(kospi_all['M_Score']==1)]))
print()
print('정상 기업 수 :',len(kospi_all[(kospi_all['M_Score']==1)&(kospi_all['target']==0)]['거래소코드'].unique()))
print('정상 데이터 수 :',len(kospi_all[(kospi_all['M_Score']==1)&(kospi_all['target']==0)]))
print()
print('부실 기업 수 :',len(kospi_all[(kospi_all['M_Score']==1)&(kospi_all['target']==1)]['거래소코드'].unique()))
print('부실 데이터 수 :',len(kospi_all[(kospi_all['M_Score']==1)&(kospi_all['target']==1)]))

<회계 부정>

부정 기업 수 : 708
부정 데이터 수 : 5273

정상 기업 수 : 708
정상 데이터 수 : 4788

부실 기업 수 : 263
부실 데이터 수 : 485


In [245]:
print('<회계 정상>\n')
print('건전 기업 수 :',len(kospi_all[(kospi_all['M_Score']==0)]['거래소코드'].unique()))
print('건전 데이터 수 :',len(kospi_all[(kospi_all['M_Score']==0)]))
print()
print('정상 기업 수 :',len(kospi_all[(kospi_all['M_Score']==0)&(kospi_all['target']==0)]['거래소코드'].unique()))
print('정상 데이터 수 :',len(kospi_all[(kospi_all['M_Score']==0)&(kospi_all['target']==0)]))
print()
print('부실 기업 수 :',len(kospi_all[(kospi_all['M_Score']==0)&(kospi_all['target']==1)]['거래소코드'].unique()))
print('부실 데이터 수 :',len(kospi_all[(kospi_all['M_Score']==0)&(kospi_all['target']==1)]))

<회계 정상>

건전 기업 수 : 49
건전 데이터 수 : 254

정상 기업 수 : 49
정상 데이터 수 : 238

부실 기업 수 : 10
부실 데이터 수 : 16


In [246]:
kospi_good = kospi_all[(kospi_all['M_Score'] == 0)]  # M1 정상
kospi_bad = kospi_all[(kospi_all['M_Score'] == 1)]  # M1 부정

kospi_good_wins = kospi_all_wins[(kospi_all_wins['M_Score'] == 0)]  # M1 정상
kospi_bad_wins = kospi_all_wins[(kospi_all_wins['M_Score'] == 1)]  # M1 부정

In [247]:
kospi_all.drop('M_Score', axis=1, inplace=True)
kospi_good.drop('M_Score', axis=1, inplace=True)
kospi_bad.drop('M_Score', axis=1, inplace=True)
kospi_all_wins.drop('M_Score', axis=1, inplace=True)
kospi_good_wins.drop('M_Score', axis=1, inplace=True)
kospi_bad_wins.drop('M_Score', axis=1, inplace=True)

#### 부도 기업 수 확인

In [114]:
#### 연도별 부도, 정상 기업 시각화

#### 연도별 데이터셋 분리

In [115]:
## golbals() -> 동적 변수 생성

In [116]:
# 변수를 생성해주기 위해 동적변수 사용 
# {괄호} 안에 변수가 생성됨
 
years = kosdaq.groupby('회계년도')
for i in kosdaq['회계년도'].unique():
    globals()['kosdaq_score_{}'.format(i)] = years.get_group(i)

In [117]:
years = kospi.groupby('회계년도')
for i in kospi['회계년도'].unique():
    globals()['kospi_score_{}'.format(i)] = years.get_group(i)

#### 데이터프레임 저장

In [118]:
kosdaq_all.to_csv('./dataset/kosdaq_all.csv', encoding='utf-8', index=False)
kosdaq_good.to_csv('./dataset/kosdaq_good.csv', encoding='utf-8', index=False)
kosdaq_bad.to_csv('./dataset/kosdaq_bad.csv', encoding='utf-8', index=False)

In [119]:
kospi_all.to_csv('./dataset/kospi_all.csv', encoding='utf-8', index=False)
kospi_good.to_csv('./dataset/kospi_good.csv', encoding='utf-8', index=False)
kospi_bad.to_csv('./dataset/kospi_bad.csv', encoding='utf-8', index=False)

In [120]:
kosdaq_all_wins.to_csv('./dataset/kosdaq_all_wins.csv', encoding='utf-8', index=False)
kosdaq_good_wins.to_csv('./dataset/kosdaq_good_wins.csv', encoding='utf-8', index=False)
kosdaq_bad_wins.to_csv('./dataset/kosdaq_bad_wins.csv', encoding='utf-8', index=False)

In [121]:
kospi_all_wins.to_csv('./dataset/kospi_all_wins.csv', encoding='utf-8', index=False)
kospi_good_wins.to_csv('./dataset/kospi_good_wins.csv', encoding='utf-8', index=False)
kospi_bad_wins.to_csv('./dataset/kospi_bad_wins.csv', encoding='utf-8', index=False)